# MSExpression Usage Guide

This notebook demonstrates how to use the refactored MSExpression class for managing gene/protein expression data in ModelSEEDpy.

## Overview

MSExpression now uses pandas DataFrame internally for efficient numerical operations while maintaining custom data structures (MSCondition, MSExpressionFeature) for metadata tracking.

**Key Features:**
- Fast vectorized operations using pandas
- Easy data loading from DataFrames, Excel, or CSV files
- Statistical analysis across conditions
- GPR-based reaction expression calculation
- Flexible data export

In [ ]:
import pandas as pd
import numpy as np
from modelseedpy.multiomics.msexpression import MSExpression
from modelseedpy.core.msgenome import MSGenome, MSFeature
from cobra import Model, Reaction, Gene

## 1. Basic Usage: Loading Expression Data

### 1.1 Creating a Sample Dataset

Let's create some sample expression data to work with:

In [ ]:
# Create sample expression data
expression_data = pd.DataFrame({
    'gene_id': ['gene1', 'gene2', 'gene3', 'gene4', 'gene5'],
    'control_1': [10.5, 15.2, 8.7, 22.1, 5.3],
    'control_2': [11.2, 14.8, 9.1, 21.5, 5.8],
    'treatment_1': [45.3, 18.2, 7.1, 35.7, 2.1],
    'treatment_2': [43.8, 19.1, 6.9, 36.2, 2.5],
    'description': ['Growth factor', 'Transporter', 'Kinase', 'Transcription factor', 'Receptor']
})

print("Sample Expression Data:")
expression_data

### 1.2 Creating a Genome with Features

In [ ]:
# Create a genome and add features
genome = MSGenome()
for gene_id in expression_data['gene_id']:
    genome.features.append(MSFeature(gene_id, ''))

print(f"Created genome with {len(genome.features)} features")

### 1.3 Loading Data from DataFrame

In [ ]:
# Load expression data from DataFrame
expr = MSExpression.from_dataframe(
    expression_data,
    genome=genome,
    id_column='gene_id',
    description_column='description',
    type='RelativeAbundance'
)

print(f"Loaded expression data:")
print(f"  - Features: {len(expr.features)}")
print(f"  - Conditions: {len(expr.conditions)}")
print(f"  - Data shape: {expr._data.shape}")

### 1.4 Accessing Expression Values

In [ ]:
# Access individual values
value = expr.get_value('gene1', 'control_1')
print(f"gene1 expression in control_1: {value}")

# Access using feature object
feature = expr.features.get_by_id('gene2')
value = feature.get_value('treatment_1')
print(f"gene2 expression in treatment_1: {value}")

## 2. Statistical Analysis

MSExpression provides efficient statistical methods for analyzing expression across conditions.

### 2.1 Basic Statistics per Condition

In [ ]:
# Get statistics for each condition
for condition in expr.conditions:
    print(f"\nStatistics for {condition.id}:")
    print(f"  Mean:    {condition.average_value():.2f}")
    print(f"  Min:     {condition.lowest_value():.2f}")
    print(f"  Max:     {condition.highest_value():.2f}")
    print(f"  Sum:     {condition.sum_value():.2f}")

### 2.2 Z-score Analysis

In [ ]:
# Calculate z-score thresholds
condition = expr.conditions.get_by_id('control_1')

z1 = condition.value_at_zscore(1.0)
z2 = condition.value_at_zscore(2.0)

print(f"Z-score thresholds for {condition.id}:")
print(f"  1σ: {z1:.2f}")
print(f"  2σ: {z2:.2f}")

### 2.3 Comparing Conditions

In [ ]:
# Compare treatment vs control for each gene
print("\nFold changes (treatment_1 / control_1):")
for feature in expr.features:
    control_val = feature.get_value('control_1')
    treatment_val = feature.get_value('treatment_1')
    
    if control_val and control_val > 0:
        fold_change = treatment_val / control_val
        print(f"  {feature.id}: {fold_change:.2f}x")

## 3. Advanced Operations

### 3.1 Working with Gene-Protein-Reaction (GPR) Rules

MSExpression can calculate reaction-level expression from gene-level data using GPR rules.

In [ ]:
# Create a simple metabolic model
model = Model('example_model')

# Add genes
for gene_id in ['gene1', 'gene2', 'gene3']:
    model.genes.append(Gene(gene_id))

# Add reactions with GPR rules
rxn1 = Reaction('rxn1')
rxn1.gene_reaction_rule = 'gene1'  # Simple single gene

rxn2 = Reaction('rxn2')
rxn2.gene_reaction_rule = 'gene1 and gene2'  # AND rule (min)

rxn3 = Reaction('rxn3')
rxn3.gene_reaction_rule = 'gene1 or gene3'  # OR rule (sum)

model.add_reactions([rxn1, rxn2, rxn3])

print(f"Created model with {len(model.reactions)} reactions")

In [ ]:
# Build reaction expression from gene expression
rxn_expr = expr.build_reaction_expression(model, default=0.0)

print("\nReaction expression values:")
for rxn_feature in rxn_expr.features:
    rxn_id = rxn_feature.id
    gpr = model.reactions.get_by_id(rxn_id).gene_reaction_rule
    control_val = rxn_feature.get_value('control_1')
    treatment_val = rxn_feature.get_value('treatment_1')
    
    print(f"\n{rxn_id} (GPR: {gpr}):")
    print(f"  Control: {control_val:.2f}")
    print(f"  Treatment: {treatment_val:.2f}")

### 3.2 Understanding GPR Logic

- **OR**: Sum of gene values (isoenzymes)
- **AND**: Minimum gene value (enzyme complex)
- **Single gene**: Direct gene value

## 4. Data Export

### 4.1 Export to DataFrame

In [ ]:
# Get DataFrame with feature_id as index (default)
df_index = expr.get_dataframe()
print("DataFrame with index:")
print(df_index.head())

In [ ]:
# Get DataFrame with feature_id as column
df_reset = expr.get_dataframe(reset_index=True)
print("\nDataFrame with reset index:")
print(df_reset.head())

### 4.2 Export to CSV/Excel

In [ ]:
# Export to CSV
df = expr.get_dataframe(reset_index=True)
# df.to_csv('expression_data.csv', index=False)

# Export to Excel
# df.to_excel('expression_data.xlsx', index=False)

print("Data can be exported using standard pandas methods")

## 5. Handling Missing Data

In [ ]:
# Create data with missing values
data_with_nan = pd.DataFrame({
    'gene_id': ['gene1', 'gene2', 'gene3'],
    'cond1': [10.0, np.nan, 15.0],
    'cond2': [20.0, 25.0, np.nan]
})

genome_test = MSGenome()
for g in ['gene1', 'gene2', 'gene3']:
    genome_test.features.append(MSFeature(g, ''))

expr_nan = MSExpression.from_dataframe(
    data_with_nan, genome=genome_test, id_column='gene_id'
)

# Missing values are returned as None
print("Handling missing values:")
print(f"gene2 in cond1: {expr_nan.get_value('gene2', 'cond1')}")  # None
print(f"gene3 in cond2: {expr_nan.get_value('gene3', 'cond2')}")  # None
print(f"gene1 in cond1: {expr_nan.get_value('gene1', 'cond1')}")  # 10.0

## 6. Migration Guide

If you're updating code from the old MSExpression implementation:

### What Changed:

1. **`get_dataframe()` format**: Now returns feature_id as index by default (use `reset_index=True` for old behavior)
2. **`feature.values` attribute**: No longer exists - values are stored in parent DataFrame

### Migration Examples:

```python
# OLD: Access feature.values directly
# value = feature.values[condition]

# NEW: Use get_value() method
value = feature.get_value(condition)

# OLD: get_dataframe() returned feature_id column
# df = expr.get_dataframe()
# df['feature_id']

# NEW: get_dataframe() has feature_id as index by default
df = expr.get_dataframe()  # feature_id is index
# Or use reset_index=True for old behavior
df = expr.get_dataframe(reset_index=True)
df['feature_id']
```

## 7. Performance Tips

The DataFrame-based implementation provides significant performance improvements:

### Best Practices:

1. **Bulk loading**: Use `from_dataframe()` instead of iterative `add_value()` calls
2. **Statistical operations**: Use MSCondition methods (they use vectorized pandas operations)
3. **Large datasets**: The DataFrame backend scales efficiently to thousands of features/conditions

### Performance Example:

In [ ]:
import time

# Create large dataset
n_genes = 5000
n_conditions = 20

large_data = pd.DataFrame(
    np.random.randn(n_genes, n_conditions + 1),
    columns=['gene_id'] + [f'cond_{i}' for i in range(n_conditions)]
)
large_data['gene_id'] = [f'gene_{i}' for i in range(n_genes)]

# Time bulk loading
large_genome = MSGenome()
for i in range(n_genes):
    large_genome.features.append(MSFeature(f'gene_{i}', ''))

start = time.time()
large_expr = MSExpression.from_dataframe(
    large_data, genome=large_genome, id_column='gene_id'
)
load_time = time.time() - start

print(f"\nPerformance Test:")
print(f"Loaded {n_genes} genes × {n_conditions} conditions in {load_time:.3f} seconds")
print(f"Data shape: {large_expr._data.shape}")

# Time statistical operations
start = time.time()
for condition in large_expr.conditions:
    avg = condition.average_value()
stats_time = time.time() - start

print(f"Computed statistics for {n_conditions} conditions in {stats_time:.3f} seconds")

## Summary

The refactored MSExpression class provides:
- ✅ Efficient DataFrame-based storage
- ✅ Fast vectorized statistical operations
- ✅ Easy data import/export
- ✅ Full backward compatibility (with minor changes)
- ✅ Type hints for better IDE support
- ✅ Comprehensive test coverage

For more information, see the ModelSEEDpy documentation.